In [1]:
import os
import pandas as pd
import numpy as np

In [11]:
#set directory
os.chdir("D:\\health_pointe")

In [12]:
#reading csv file
item=pd.read_csv("combined_events.csv",delimiter='|')

In [38]:
item.head(5)

,START_TS,PATIENT_ID,ENCOUNTER_ID,ENCOUNTER_CODE,ENCOUNTER_DATE,ENCOUNTER_DESC,EVENT_TYPE,EVENT_AGE,EVENT_CODE,EVENT_DESC,END_TS,new_date
0,2007-11-28,81979c08-df99-4792-b608-65fcf28f4019,62af7c80-a6b7-4ffa-b495-677c9fc24aec,185345009,2007-12-08,Encounter for symptom,CONDITION,CURRENT,444814009,Viral sinusitis (disorder),2007-12-28,2007
1,2008-06-01,81979c08-df99-4792-b608-65fcf28f4019,fd1d755d-8894-42d5-afd9-a4fd114e1e85,185345009,2008-06-05,Encounter for symptom,CONDITION,CURRENT,195662009,Acute viral pharyngitis (disorder),2008-06-15,2008
2,2010-06-14,81979c08-df99-4792-b608-65fcf28f4019,f3c9bdc5-2474-4458-a2cd-3255353a21e5,185349003,2010-06-14,Outpatient Encounter,CONDITION,CURRENT,38341003,Hypertension,NaN,2010
3,2010-06-14,81979c08-df99-4792-b608-65fcf28f4019,f3c9bdc5-2474-4458-a2cd-3255353a21e5,185349003,2010-06-14,Outpatient Encounter,CONDITION,CURRENT,44054006,Diabetes,NaN,2010
4,2013-12-28,81979c08-df99-4792-b608-65fcf28f4019,043a8aba-ebb6-4095-ac17-2b30c72e4407,185345009,2014-01-03,Encounter for symptom,CONDITION,CURRENT,444814009,Viral sinusitis (disorder),2014-01-13,2014


In [13]:
#change data type and doing required transformation

item['new_date']=pd.to_datetime(item['ENCOUNTER_DATE'])
item['new_date'] = item['new_date'].dt.strftime('%Y')

item['new_date']=item['new_date'].astype('str')

In [21]:
#group by patient_id with visit years
combine_fea1=item.groupby('PATIENT_ID')['new_date'].apply(" ".join).reset_index()

In [22]:
combine_fea1

,PATIENT_ID,new_date
0,280c458c-b8f0-4533-b7d1-3b2ac12fe950,2014 2015 2017
1,6cd45b81-284d-48b8-bab1-a4e65784d813,2011
2,81979c08-df99-4792-b608-65fcf28f4019,2007 2008 2010 2010 2014 2016
3,b350b118-f30d-4a9d-a006-56be45a4da13,2013 2015 2015 2016 2017
4,db6649b6-9957-4904-9f49-b9270fda2410,2007 2009 2009 2013 2015
5,ee7bc529-f806-4510-b5dc-016a4eb8fe36,2008 2009 2009 2009 2011 2013 2013 2015 2015 2016


In [23]:
#combine all conditions patients have
combine_fea2=item.groupby('PATIENT_ID')['EVENT_DESC'].apply(" ".join).reset_index()

In [24]:
#adding information from previous data frame
combine_fea2['CONDITION_EVENT_YEAR']=combine_fea1['new_date']

In [25]:
combine_fea2

,PATIENT_ID,EVENT_DESC,CONDITION_EVENT_YEAR
0,280c458c-b8f0-4533-b7d1-3b2ac12fe950,Otitis media Otitis media Otitis media,2014 2015 2017
1,6cd45b81-284d-48b8-bab1-a4e65784d813,Alzheimer's disease (disorder),2011
2,81979c08-df99-4792-b608-65fcf28f4019,Viral sinusitis (disorder) Acute viral pharyng...,2007 2008 2010 2010 2014 2016
3,b350b118-f30d-4a9d-a006-56be45a4da13,Fracture subluxation of wrist Acute bronchitis...,2013 2015 2015 2016 2017
4,db6649b6-9957-4904-9f49-b9270fda2410,Fracture of rib Streptococcal sore throat (dis...,2007 2009 2009 2013 2015
5,ee7bc529-f806-4510-b5dc-016a4eb8fe36,Facial laceration Acute bronchitis (disorder) ...,2008 2009 2009 2009 2011 2013 2013 2015 2015 2016


In [26]:
# copying complete info to new data frame and deleting columns which are not required.

new_item=item.copy()
var_new=['ENCOUNTER_ID', 'ENCOUNTER_CODE','ENCOUNTER_DESC', 'EVENT_TYPE','EVENT_DESC']
new_item=new_item.drop(var_new,axis=1)

In [27]:
import datetime as dt
#changing feature types and doing required transformations
new_item['ENCOUNTER_DATE']=pd.to_datetime(new_item['ENCOUNTER_DATE'])
new_item['Date1'] = new_item['ENCOUNTER_DATE'].dt.strftime('%m-%d')

new_item['START_TS']=pd.to_datetime(new_item['START_TS'])
new_item['Date2'] = new_item['START_TS'].dt.strftime('%m-%d')

new_item['END_TS']=pd.to_datetime(new_item['END_TS'])
new_item['Date3'] = new_item['END_TS'].dt.strftime('%m-%d')


In [28]:
new_item['EVENT_CODE']=new_item['EVENT_CODE'].astype('str')
new_item['Date1']=new_item['Date1'].astype('str')
new_item['Date2']=new_item['Date2'].astype('str')
new_item['Date3']=new_item['Date3'].astype('str')

In [29]:
#combining all in one.
new_item["CONDITION_CODE_DATE"] = new_item["EVENT_CODE"] +":" + new_item["Date1"] +"/" + new_item["Date2"] +"/" + new_item["Date3"]


In [30]:
new_item.head(2)

,START_TS,PATIENT_ID,ENCOUNTER_DATE,EVENT_AGE,EVENT_CODE,END_TS,new_date,Date1,Date2,Date3,CONDITION_CODE_DATE
0,2007-11-28,81979c08-df99-4792-b608-65fcf28f4019,2007-12-08,CURRENT,444814009,2007-12-28,2007,12-08,11-28,12-28,444814009:12-08/11-28/12-28
1,2008-06-01,81979c08-df99-4792-b608-65fcf28f4019,2008-06-05,CURRENT,195662009,2008-06-15,2008,06-05,06-01,06-15,195662009:06-05/06-01/06-15


In [31]:
##combine all CONDITION_CODE_DATE created above as per patients id.


combine_fea3=new_item.groupby('PATIENT_ID')['CONDITION_CODE_DATE'].apply(' '.join).reset_index()

In [32]:
combine_fea3

,PATIENT_ID,CONDITION_CODE_DATE
0,280c458c-b8f0-4533-b7d1-3b2ac12fe950,65363002:12-30/12-30/03-22 65363002:10-22/10-2...
1,6cd45b81-284d-48b8-bab1-a4e65784d813,26929004:04-21/04-21/NaT
2,81979c08-df99-4792-b608-65fcf28f4019,444814009:12-08/11-28/12-28 195662009:06-05/06...
3,b350b118-f30d-4a9d-a006-56be45a4da13,263102004:07-20/07-20/10-14 10509002:02-20/02-...
4,db6649b6-9957-4904-9f49-b9270fda2410,33737001:12-23/12-23/04-14 43878008:08-15/08-1...
5,ee7bc529-f806-4510-b5dc-016a4eb8fe36,370247008:02-05/02-05/02-24 10509002:01-17/01-...


In [33]:
#creating final data frame by merging all information required.
final= pd.merge(combine_fea2,combine_fea3,on='PATIENT_ID')


In [34]:
final

,PATIENT_ID,EVENT_DESC,CONDITION_EVENT_YEAR,CONDITION_CODE_DATE
0,280c458c-b8f0-4533-b7d1-3b2ac12fe950,Otitis media Otitis media Otitis media,2014 2015 2017,65363002:12-30/12-30/03-22 65363002:10-22/10-2...
1,6cd45b81-284d-48b8-bab1-a4e65784d813,Alzheimer's disease (disorder),2011,26929004:04-21/04-21/NaT
2,81979c08-df99-4792-b608-65fcf28f4019,Viral sinusitis (disorder) Acute viral pharyng...,2007 2008 2010 2010 2014 2016,444814009:12-08/11-28/12-28 195662009:06-05/06...
3,b350b118-f30d-4a9d-a006-56be45a4da13,Fracture subluxation of wrist Acute bronchitis...,2013 2015 2015 2016 2017,263102004:07-20/07-20/10-14 10509002:02-20/02-...
4,db6649b6-9957-4904-9f49-b9270fda2410,Fracture of rib Streptococcal sore throat (dis...,2007 2009 2009 2013 2015,33737001:12-23/12-23/04-14 43878008:08-15/08-1...
5,ee7bc529-f806-4510-b5dc-016a4eb8fe36,Facial laceration Acute bronchitis (disorder) ...,2008 2009 2009 2009 2011 2013 2013 2015 2015 2016,370247008:02-05/02-05/02-24 10509002:01-17/01-...
